# Development scratchbook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import etfs as et

In [ ]:
from etfs.security.io import read_yahoo_csv, retrieve_yahoo_quote, get_company_name
from etfs.stats.basics import resample_df, runrate_column, shift_column, returns_column, \
                              difference, rsq, beta, alpha, standard_deviation_column
from etfs.utils.helpers import todays_date
from etfs.visualization.trends import plot_trend, plot_candlestick, plot_ewm
from etfs.treasury.io import retrieve_treasury_yield_curve_rates

## Retrieve quotes from Yahoo! Finance

In [ ]:
df = retrieve_yahoo_quote(ticker='VO', startdate='19000101', enddate='20180204')

In [ ]:
df.head()

## Alternatively: read in files

In [ ]:
filepath = '../data/VO.csv'

In [ ]:
df = read_yahoo_csv(path=filepath)

In [ ]:
df.head(5)

## First look at the data

In [ ]:
df[['Close', 'Adj Close']].plot()

## Compare several securities

In [ ]:
#filepath = '../data/VOO.csv'
#df2 = read_yahoo_csv(path=filepath)
df2 = retrieve_yahoo_quote(ticker='VOO', enddate=todays_date())

In [ ]:
#filepath = '../data/VNQ.csv'
#df3 = read_yahoo_csv(path=filepath)
df3 = retrieve_yahoo_quote(ticker='VNQ', enddate=todays_date())

In [ ]:
plt.plot(df.index, df.Close, label='VO')
plt.plot(df2.index, df2.Close, label = 'VOO')
plt.plot(df3.index, df3.Close, label='VNQ')
plt.legend()

## Play with some basic time series functions

### Run rate

In [ ]:
df = runrate_column(df, column='Close', window=50)
df = runrate_column(df, column='Close', window=200)

In [ ]:
plt.plot(df.index, df.Close)
plt.plot(df.index, df.Close_rr50)
plt.plot(df.index, df.Close_rr200)

### Resampling

In [ ]:
df_m = resample_df(df, column='Close', resolution='M')
df_y = resample_df(df, column='Close', resolution='12M')

In [ ]:
plt.plot(df.index, df.Close)
plt.plot(df_m.index, df_m.Close['mean'])
plt.plot(df_y.index, df_y.Close['mean'])

In [ ]:
df_y.head(5)

### Volatility

In [ ]:
df = standard_deviation_column(df, column='Close', window=1, shift=1)

In [ ]:
df2 = standard_deviation_column(df2, column='Close', window=1, shift=1)

In [ ]:
df3 = standard_deviation_column(df3, column='Close', window=1, shift=1)

In [ ]:
df_m = resample_df(df, column='Close_sh1_sqerr', resolution='7D')
df2_m = resample_df(df2, column='Close_sh1_sqerr', resolution='7D')
df3_m = resample_df(df3, column='Close_sh1_sqerr', resolution='7D')

In [ ]:
plt.plot(df3_m.index, np.sqrt(df3_m.Close_sh1_sqerr['mean'])+.2, label='VNQ')
plt.plot(df_m.index, np.sqrt(df_m.Close_sh1_sqerr['mean'])+.1, label='VO')
plt.plot(df2_m.index, np.sqrt(df2_m.Close_sh1_sqerr['mean']), label='VOO')
plt.legend()
plt.ylabel('Volatility')
plt.yticks([]);

## Differences in time

In [ ]:
startdate = '2017-01-03'
enddate   = '2017-02-09'

In [ ]:
df_plot = df.loc[(df.index >= startdate) & (df.index <= enddate), :]
plt.plot(df_plot.index, df_plot.Close)

In [ ]:
difference(df=df, column='Close', start=startdate, end=enddate)

## Security class

In [ ]:
VO = et.Security('VO', start='2017-01-01', end='2017-12-31')
VOO = et.Security('VOO', start='2017-01-01', end='2017-12-31')
VNQ = et.Security('VNQ', start='2017-01-01', end='2017-12-31')

In [ ]:
VO.ticker, VOO.ticker, VNQ.ticker

In [ ]:
VNQ.refresh()

### Load data either from file or from Yahoo!

In [ ]:
VO.load(start='2017-01-01', end='2017-12-31')
VOO.load(start='2017-01-01', end='2017-12-31')

In [ ]:
difference(df=VO.data, column='Close', start='2017-01-01', end='2017-01-30')

In [ ]:
difference(df=VOO.data, column='Close', start='2017-01-01', end='2017-01-30')

### Name class instance

In [ ]:
VO.name = 'Vanguard Mid Cap ETF'

In [ ]:
VO.name

In [ ]:
VO.set_name('VO')

In [ ]:
VO.name

In [ ]:
print(VO)

### Last price in data

In [ ]:
VO.get_last_price('Close')

In [ ]:
VO.last_price

### Plot outliers

In [ ]:
df_high = VNQ.data.loc[VNQ.data.Close >= VNQ.mean_price + VNQ.std_price]
df_low = VNQ.data.loc[VNQ.data.Close <= VNQ.mean_price - VNQ.std_price]

In [ ]:
plt.plot(VNQ.data.index, VNQ.data.Close)
plt.scatter(df_high.index, df_high.Close)
plt.scatter(df_low.index, df_low.Close)

### Refresh data (and save as csv)

In [ ]:
VO.refresh()

In [ ]:
VOO.refresh()

### Calculate returns

In [ ]:
returns_column(df=VO.data, column='Close').head()

In [ ]:
VO.get_returns()

In [ ]:
VO.data.Return.hist()

In [ ]:
VOO.get_returns()

In [ ]:
VOO.data.Return.hist()

### R squared between two securities

In [ ]:
rsq(sec1=VOO, sec2=VO, col1='Close', col2='Close')

In [ ]:
rsq(sec1=VNQ, sec2=VO, col1='Close', col2='Close')

In [ ]:
VO.get_returns()

In [ ]:
VOO.get_returns()

In [ ]:
VNQ.get_returns()

In [ ]:
rsq(sec1=VOO, sec2=VO, col1='Return', col2='Return')

In [ ]:
rsq(sec1=VNQ, sec2=VO, col1='Return', col2='Return')

### Calculate beta of a security against a benchmark

In [ ]:
VO.get_benchmark()

In [ ]:
VO.benchmark.get_returns()

In [ ]:
beta(sec1=VO, sec2=VO.benchmark, col1='Return', col2='Return')

In [ ]:
VOO.get_benchmark()

In [ ]:
VOO.benchmark.get_returns()

In [ ]:
beta(sec1=VOO, sec2=VOO.benchmark, col1='Return', col2='Return')

### Calculate alpha of a security against a benchmark

In [ ]:
VO = et.Security('VO', start='2015-01-01', end='2017-12-31')

In [ ]:
VO.get_benchmark()

In [ ]:
VO.get_returns()

In [ ]:
alpha(sec1=VO, sec2=VO.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

### 50-day and 200-day rolling weighted averages for securities 

In [ ]:
NFLX = et.Security('NFLX')

In [ ]:
plot_trend(security=NFLX, windows=[50, 200], ndays=1000)

In [ ]:
AMD = et.Security('AMD')

In [ ]:
plot_trend(security=AMD, windows=[10, 30])

In [ ]:
BIDU = et.Security('BIDU')

In [ ]:
plot_trend(security=BIDU, windows=[50, 200], ndays=1000)

In [ ]:
BABA = et.Security('BABA')

In [ ]:
plot_trend(security=BABA, windows=[50, 200], ndays=1000)

In [ ]:
W = et.Security('W')

In [ ]:
plot_trend(security=W, ndays=1000)

In [ ]:
SP500 = et.Security('^GSPC')

In [ ]:
plot_trend(security=SP500, windows=[50, 200])

In [ ]:
IQ = et.Security('IQ')

In [ ]:
plot_trend(security=IQ, windows=[50, 200])

In [ ]:
JD = et.Security('JD')

In [ ]:
plot_trend(security=JD, windows=[50, 200])

In [ ]:
SONO = et.Security('SONO', start='2018-01-01')

In [ ]:
plot_trend(security=SONO, windows=[20, 50])

In [ ]:
VOO = et.Security('VOO', start='2018-01-01')

In [ ]:
plot_trend(security=VOO, windows=[50, 200])

In [ ]:
SNE = et.Security('SNE')

In [ ]:
plot_trend(security=SNE, windows=[50, 200], ndays=1000)

### Exponentially weighted moving averages

In [ ]:
FB = et.Security('FB')

In [ ]:
plot_ewm(security=FB, alphas=[.5, .1])

In [ ]:
plot_trend(security=FB, windows=[10, 30])

### Candlestick plot

In [ ]:
AAPL = et.Security('AAPL', start='2008-01-02', end='2017-12-29')

In [ ]:
AAPL.data

In [ ]:
AAPL.data.index.max()

In [ ]:
plot_candlestick(security=AAPL, 
                 open_col='Open',
                 close_col='Close',
                 high_col='High',
                 low_col='Low',
                 ndays=100,
                 heikin=False
                )

### Heikin-Ashi candlestick plot

In [ ]:
plot_candlestick(security=AAPL, 
                 open_col='Open',
                 close_col='Close',
                 high_col='High',
                 low_col='Low',
                 ndays=100,
                 heikin=True
                )

## Get Treasury Yield Curve rates

In [ ]:
df = retrieve_treasury_yield_curve_rates(startdate='20010101', enddate='20190105')

In [ ]:
UST = et.Treasury(name='UST', start='2001-01-01')

In [ ]:
plt.plot(UST.data.index, UST.data['3 mo'])
plt.plot(UST.data.index, UST.data['1 yr'])
plt.legend()
plt.ylabel('')
plt.title('U.S. Treasuries yield rates [%]', loc='left', fontsize=12)
sns.despine(offset=10)
plt.tight_layout()

In [ ]:
plt.plot(UST.data.index, UST.data['3 mo']/UST.data['1 yr'])
plt.hlines([1], xmin=UST.data.index.min(), xmax=UST.data.index.max(), linestyles='--')
plt.ylabel('')
plt.title('Ratio of U.S. Treasuries yield rates 3 mo / 1 yr', loc='left', fontsize=12)
sns.despine(offset=10)
plt.tight_layout()


## Portfolios

In [ ]:
from etfs.portfolio.io import import_portfolio

In [ ]:
test = et.Portfolio(name="test portfolio")

In [ ]:
test = import_portfolio(path="../data/portfolio_test.csv", name="Test Portfolio")

In [ ]:
test.overview()

In [ ]:
test.positions()

In [ ]:
test.get_timeseries()

In [ ]:
test.data_growth.plot(y='Growth')

In [ ]:
test.data_growth.loc['2018-06-4', :]